In [1]:
import numpy as np
import matplotlib.pyplot as plt
from music21.midi import MidiFile 
from music21.midi.translate import getTimeForEvents

import itertools

In [2]:
from modules.midiscoretools import extract_time_signatures, ticks_per_beat_at_tick
from modules.midiscoretools import beats_per_measure_at_tick, time2tick, tick2time
from modules.midiscoretools import count_total_ticks, Frame

In [3]:
from programs.CreateRefData import float_range,  midi2frameskeleton, process_json_loc_file, measureBeats2frameList

<div style="text-align:center; margin: 2px 0;">
    <h2 style="display:inline-block; padding: 0 10px; font-family: 'Brush Script MT', cursive; font-size: 16px; color:#1e90ff;">
        ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
        <span style="color:#800000; font-family: 'Georgia', serif; font-size: 18px; font-weight: bold; padding: 0 15px;">Parameters</span>
        ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿ ✿
    </h2>
</div>

In [4]:
midi_file="scores/Test3/RefData/Test3.mid"
fps=86.1328125

# generated by the MuseScore plug-in
jsonmbinfo_file="scores/Test3/RefData/Test3.locinfo.jsn" #"scratch2/Test3.beatinfo2.jsn"


In [5]:
######################
#  just check a few things for fun ... these variables are not needed later
######################
if False: 
    mf = MidiFile()
    mf.open(midi_file)
    mf.read()
    PPQ=mf.ticksPerQuarterNote
    tracks = range(len(mf.tracks))
    print(f' num tracks = {len(mf.tracks)}, and PPQ = {PPQ}')
    print(f'  PPQ = {PPQ}')
    
    #just check the utility
    print(f' time2tick {time2tick(mf, 27.7125)}')
    print(f' tick2time {tick2time(mf, 10308)}')
    print(f' there and back {time2tick(mf, tick2time(mf, 10308))}')
    
    print(f' Total ticks in midi_file:  {count_total_ticks(midi_file)}')
    
    print(Frame.__doc__)

<span style="color:#800000;  font-size: 16px; font-weight: bold;  ">midi2frameskeleton</span> (creates a list of equally spaced frames in time with the tick values of their middles and endpoints)

In [6]:
refFrameList=midi2frameskeleton(midi_file,fps)
print(refFrameList[0:3])
print("...")
print(refFrameList[-3:])

[Frame(0, sTk=0, sTm=0, mTk=5, mTm=0.006, measure=None, beat=None, refframe=0), Frame(1, sTk=11, sTm=0.012, mTk=16, mTm=0.017, measure=None, beat=None, refframe=1), Frame(2, sTk=22, sTm=0.023, mTk=27, mTm=0.029, measure=None, beat=None, refframe=2)]
...
[Frame(2384, sTk=10297, sTm=27.678, mTk=10298, mTm=27.684, measure=None, beat=None, refframe=2384), Frame(2385, sTk=10300, sTm=27.69, mTk=10302, mTm=27.696, measure=None, beat=None, refframe=2385), Frame(2386, sTk=10304, sTm=27.701, mTk=10306, mTm=27.707, measure=None, beat=None, refframe=2386)]


<span style="color:#800000;  font-size: 16px; font-weight: bold;  ">get the measure/beat location info</span> Read in the time stamped events from the musicXML plugin (including measures that repeat)

In [7]:
tmb=process_json_loc_file(jsonmbinfo_file)

<span style="color:#800000;  font-size: 16px; font-weight: bold;  ">reality check printout</span> 

In [8]:
######################
#  just check a few things for fun ... these variables are not needed later
######################
if False: 
    mf = MidiFile()
    mf.open(midi_file)
    mf.read()
    
    prevTick=0
    for i, b in enumerate(tmb) :
        if i < 0 or i > len(tmb)-1 :
            print(f'{i} : {b}, ticks={time2tick(mf, b[0])}')
            prevTick=time2tick(mf, b[0])
        else :
            print(f'{i} : {b}, ticks={time2tick(mf, b[0])}, delta={time2tick(mf, b[0]) - prevTick}')
            prevTick=time2tick(mf, b[0])

<span style="color:#800000;  font-size: 16px; font-weight: bold;  ">put measure and beats in to frames</span> first from the MuseScore plugin (all we really need from there is info about measures to track repeats!), and then fill in frames between. 

In [9]:
#Create a list of the time signatures so that it can be used to calculate beat info
# verbose shows assignments from the tmb measure beat list
frames=measureBeats2frameList(midi_file, tmb, refFrameList, verbose=True)

assigned mb[0] at 0 to frame(0, mTk=5, mTm=0.006, measure=0, beat=1)
assigned mb[1] at 0.25 to frame(21, mTk=239, mTm=0.25, measure=0, beat=2)
assigned mb[2] at 0.5 to frame(43, mTk=484, mTm=0.505, measure=0, beat=3)
assigned mb[3] at 0.75 to frame(64, mTk=718, mTm=0.749, measure=1, beat=1)
assigned mb[4] at 1 to frame(86, mTk=964, mTm=1.004, measure=1, beat=2)
assigned mb[5] at 1.25 to frame(107, mTk=1198, mTm=1.248, measure=1, beat=3)
assigned mb[6] at 1.5 to frame(129, mTk=1443, mTm=1.503, measure=1, beat=4)
assigned mb[7] at 1.75 to frame(150, mTk=1677, mTm=1.747, measure=1, beat=5)
assigned mb[8] at 2 to frame(172, mTk=1922, mTm=2.003, measure=1, beat=6)
assigned mb[9] at 2.25 to frame(193, mTk=2156, mTm=2.247, measure=2, beat=1)
assigned mb[10] at 3 to frame(258, mTk=2400, mTm=3.001, measure=2, beat=2)
assigned mb[11] at 3.75 to frame(322, mTk=2638, mTm=3.744, measure=2, beat=3)
assigned mb[12] at 4.5 to frame(387, mTk=2879, mTm=4.499, measure=2, beat=4)
assigned mb[13] at 5.25 t

In [10]:
#show all frames with beat interpolations
frames[:86]


[Frame(0, sTk=0, sTm=0, mTk=5, mTm=0.006, measure=0, beat=1, refframe=0),
 Frame(1, sTk=11, sTm=0.012, mTk=16, mTm=0.017, measure=0, beat=1.0458333333333334, refframe=1),
 Frame(2, sTk=22, sTm=0.023, mTk=27, mTm=0.029, measure=0, beat=1.0916666666666668, refframe=2),
 Frame(3, sTk=33, sTm=0.035, mTk=39, mTm=0.041, measure=0, beat=1.1416666666666668, refframe=3),
 Frame(4, sTk=44, sTm=0.046, mTk=50, mTm=0.052, measure=0, beat=1.1875000000000002, refframe=4),
 Frame(5, sTk=55, sTm=0.058, mTk=61, mTm=0.064, measure=0, beat=1.2333333333333336, refframe=5),
 Frame(6, sTk=66, sTm=0.07, mTk=72, mTm=0.075, measure=0, beat=1.279166666666667, refframe=6),
 Frame(7, sTk=78, sTm=0.081, mTk=83, mTm=0.087, measure=0, beat=1.3250000000000004, refframe=7),
 Frame(8, sTk=89, sTm=0.093, mTk=94, mTm=0.099, measure=0, beat=1.3708333333333338, refframe=8),
 Frame(9, sTk=100, sTm=0.104, mTk=105, mTm=0.11, measure=0, beat=1.4166666666666672, refframe=9),
 Frame(10, sTk=111, sTm=0.116, mTk=117, mTm=0.122, mea

In [11]:
frames[-86:]

[Frame(2301, sTk=9988, sTm=26.715, mTk=9990, mTm=26.72, measure=6, beat=5.633333333333334, refframe=2301),
 Frame(2302, sTk=9992, sTm=26.726, mTk=9994, mTm=26.732, measure=6, beat=5.65, refframe=2302),
 Frame(2303, sTk=9996, sTm=26.738, mTk=9997, mTm=26.744, measure=6, beat=5.6625000000000005, refframe=2303),
 Frame(2304, sTk=9999, sTm=26.749, mTk=10001, mTm=26.755, measure=6, beat=5.679166666666667, refframe=2304),
 Frame(2305, sTk=10003, sTm=26.761, mTk=10005, mTm=26.767, measure=6, beat=5.695833333333334, refframe=2305),
 Frame(2306, sTk=10007, sTm=26.773, mTk=10009, mTm=26.778, measure=6, beat=5.7125, refframe=2306),
 Frame(2307, sTk=10010, sTm=26.784, mTk=10012, mTm=26.79, measure=6, beat=5.7250000000000005, refframe=2307),
 Frame(2308, sTk=10014, sTm=26.796, mTk=10016, mTm=26.802, measure=6, beat=5.741666666666667, refframe=2308),
 Frame(2309, sTk=10018, sTm=26.807, mTk=10020, mTm=26.813, measure=6, beat=5.758333333333334, refframe=2309),
 Frame(2310, sTk=10022, sTm=26.819, mTk=1

In [13]:
27.71*86.1328

2386.739888